In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np

In [2]:
june = pd.read_pickle('../data/june.pkl')
junetrip = pd.read_csv('../data/june_trip.csv')

In [3]:
june.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
0,2019-06-01 00:00:12,36.1202,-86.7534,Powered93627c35-0f62-5b81-a78d-75a4a92ecf47,90.0,5
1,2019-06-01 00:00:12,36.1630,-86.7765,Powered17715097-e8a0-5494-a5ab-9b625796607d,63.0,5
2,2019-06-01 00:00:12,36.1202,-86.7533,Powerede5cb95ae-b091-5a93-86fa-ededd946d0d7,77.0,5
3,2019-06-01 00:00:12,36.1201,-86.7530,Powered71fa5e4f-1e17-54c4-936d-330df38cc2fa,0.0,5
4,2019-06-01 00:00:12,36.1622,-86.7806,Poweredfa549dd6-40bb-5757-ac87-2c2528f2bc68,2.0,5


In [4]:
june.tail()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
28046090,2019-06-30 23:59:55.273,36.125567,-86.776982,PoweredZYNSVOII2QSLD,100.0,3
28046091,2019-06-30 23:59:55.273,36.121717,-86.775207,PoweredXOPAIWGL7SF62,93.0,3
28046092,2019-06-30 23:59:55.273,36.127046,-86.777262,PoweredBYKFOWELGPKH6,61.0,3
28046093,2019-06-30 23:59:55.273,36.123775,-86.790115,PoweredIAAQ7WODYXON7,35.0,3
28046094,2019-06-30 23:59:55.273,36.121735,-86.775148,Powered3HJPPUNVYGL45,96.0,3


In [5]:
june.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28037408 entries, 0 to 28046094
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   chargelevel  float64       
 5   companyname  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 1.5+ GB


#note: there are 8 million more rows of data in June than there are in May.  The month is actually shorter than may, so it's not that ... Maybe more logging per scooter?  maybe more scooters in total?

In [6]:
june['sumdid'].value_counts() 

PoweredUNORTWSTRM4AM                           8631
PoweredBV42LPYJGVBL4                           8630
Poweredec02467e-2602-5e91-a74d-33f6f53a0c07    8628
Poweredb76d8a78-2288-548e-8008-657447cec609    8628
Powered97cf10e2-9da0-55d2-bd92-2637a76e39cb    8628
                                               ... 
Powered911DI                                      1
Powered255293                                     1
Powered8112481                                    1
Powered2WZRP                                      1
PoweredUKDFR5O5HBMBO                              1
Name: sumdid, Length: 7822, dtype: int64

nope - fewer scooters.  there were 8303 in may, there's 7822 in june.
I still see the top number of entries is about the same, and the bottom is still 1

In [7]:
june['companyname'].value_counts()

5    8482893
3    6152504
0    4215302
1    3084229
4    2507991
2    1830453
6    1764036
Name: companyname, dtype: int64

interesting - in may, company 3 had the most entries.  in june, it's company 5.  they stepped up their data collection.  do they have more scooters too?  Even though the overall number is smaller?

In [8]:
june_company_group = june.groupby("companyname")

In [9]:
june_scooter_company = june_company_group.agg({"sumdid": "nunique"})
print(june_scooter_company)

             sumdid
companyname        
0              2910
1              1152
2               223
3              1455
4               713
5              1036
6               333


company 0 dropped 154 scooters;
company 1 dropped 224 scooters;
company 2 same;
company 3 dropped 176 scooters;
company 4 gained 26 scooters;
company 5 gained 60 scooters;
company 6 dropped 13 scooters

In [10]:
june.isnull().sum()

pubdatetime      0
latitude         0
longitude        0
sumdid           0
chargelevel    285
companyname      0
dtype: int64

In [11]:
june_nullcharge_company= june.chargelevel.isnull().groupby(june.companyname).sum()
print(june_nullcharge_company)

companyname
0      0
1      0
2      0
3      0
4      0
5      0
6    285
Name: chargelevel, dtype: int64


i was careful to make sure that the company name/numbers were the same across all 3 months.  May all nulls were in company 4.   june, all nulls are in company 6.  what changed?

just for kicks, let's see how many single value entries there are in june ... 

In [12]:
june.groupby("sumdid").filter(lambda x: len(x) == 1)

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
3045,2019-06-01 00:03:27.257,36.157671,-86.773311,PoweredNIBNX6HK6AP5L,31.0,3
639001,2019-06-01 16:40:58.923,36.082328,-86.725584,Powered911DI,78.0,0
4372676,2019-06-05 13:02:07.010,36.147920,-86.740430,Powered8112481,0.0,4
11366205,2019-06-12 20:58:57.477,36.137113,-86.800817,PoweredUKDFR5O5HBMBO,20.0,3
16974731,2019-06-18 22:16:16.873,36.213138,-86.735270,Powered2WZRP,62.0,0
18752982,2019-06-20 17:12:50.910,36.213265,-86.735226,PoweredXCPDB,48.0,0
23512054,2019-06-26 03:50:10.600,36.149604,-86.797419,Powered6668448,92.0,4
28043947,2019-06-30 23:56:15.457,36.177585,-86.750157,Powered255293,91.0,1


just 8 this time.  off hand, looks like different scooters  similar distribution across companies, dates, locations ... 

repeating what i learned playing in may's data to june

In [13]:
june['dates'] = pd.to_datetime(june['pubdatetime']).dt.date
june['times'] = pd.to_datetime(june['pubdatetime']).dt.time
june['hours'] = pd.to_datetime(june['pubdatetime']).dt.hour

In [14]:
june.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,dates,times,hours
0,2019-06-01 00:00:12,36.1202,-86.7534,Powered93627c35-0f62-5b81-a78d-75a4a92ecf47,90.0,5,2019-06-01,00:00:12,0
1,2019-06-01 00:00:12,36.1630,-86.7765,Powered17715097-e8a0-5494-a5ab-9b625796607d,63.0,5,2019-06-01,00:00:12,0
2,2019-06-01 00:00:12,36.1202,-86.7533,Powerede5cb95ae-b091-5a93-86fa-ededd946d0d7,77.0,5,2019-06-01,00:00:12,0
3,2019-06-01 00:00:12,36.1201,-86.7530,Powered71fa5e4f-1e17-54c4-936d-330df38cc2fa,0.0,5,2019-06-01,00:00:12,0
4,2019-06-01 00:00:12,36.1622,-86.7806,Poweredfa549dd6-40bb-5757-ac87-2c2528f2bc68,2.0,5,2019-06-01,00:00:12,0


In [15]:
#june.loc[june["dates"]== datetime.date(2019, 6, 1)]
#to filter my df by a date

In [16]:
june.loc[june["hours"]==(7)]
#to look at a specific hour window

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,dates,times,hours
264180,2019-06-01 07:00:13.000,36.120200,-86.753600,Powered89337779-ccd2-5d97-86a0-8b45c15f8c43,0.0,5,2019-06-01,07:00:13,7
264181,2019-06-01 07:00:13.000,36.119700,-86.753000,Poweredb90ec96b-f1e7-5fd5-bed4-d98d1ebeb7dc,0.0,5,2019-06-01,07:00:13,7
264182,2019-06-01 07:00:13.000,36.119600,-86.753400,Poweredb4d3abf4-8f62-54b6-af45-b424c70370f2,4.0,5,2019-06-01,07:00:13,7
264183,2019-06-01 07:00:13.000,36.120300,-86.753600,Poweredcc2901ac-99d2-5738-b185-120bd22bc743,88.0,5,2019-06-01,07:00:13,7
264184,2019-06-01 07:00:13.000,36.154200,-86.798700,Powereda51ff7e4-57b3-53ac-ad40-02aad9208d64,72.0,5,2019-06-01,07:00:13,7
...,...,...,...,...,...,...,...,...,...
27458218,2019-06-30 07:59:18.957,36.110709,-86.768675,PoweredOH5ERPUT5P5PH,71.0,3,2019-06-30,07:59:18.957000,7
27458219,2019-06-30 07:59:18.957,36.126544,-86.790246,PoweredQEL7BQWGOKPCP,86.0,3,2019-06-30,07:59:18.957000,7
27458220,2019-06-30 07:59:18.957,36.120400,-86.775460,PoweredJB4Y6ZOOR4IBC,76.0,3,2019-06-30,07:59:18.957000,7
27458221,2019-06-30 07:59:18.957,36.120568,-86.775256,PoweredC7MTAEJR5C2Y7,86.0,3,2019-06-30,07:59:18.957000,7


In [17]:
june.loc[(june['hours'] > 6) & (june['hours'] < 10)]
#a range of time - the morning commute

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,dates,times,hours
264180,2019-06-01 07:00:13.000,36.120200,-86.753600,Powered89337779-ccd2-5d97-86a0-8b45c15f8c43,0.0,5,2019-06-01,07:00:13,7
264181,2019-06-01 07:00:13.000,36.119700,-86.753000,Poweredb90ec96b-f1e7-5fd5-bed4-d98d1ebeb7dc,0.0,5,2019-06-01,07:00:13,7
264182,2019-06-01 07:00:13.000,36.119600,-86.753400,Poweredb4d3abf4-8f62-54b6-af45-b424c70370f2,4.0,5,2019-06-01,07:00:13,7
264183,2019-06-01 07:00:13.000,36.120300,-86.753600,Poweredcc2901ac-99d2-5738-b185-120bd22bc743,88.0,5,2019-06-01,07:00:13,7
264184,2019-06-01 07:00:13.000,36.154200,-86.798700,Powereda51ff7e4-57b3-53ac-ad40-02aad9208d64,72.0,5,2019-06-01,07:00:13,7
...,...,...,...,...,...,...,...,...,...
27511314,2019-06-30 09:59:23.120,36.125994,-86.789209,PoweredINKMXRRD3D3SC,98.0,3,2019-06-30,09:59:23.120000,9
27511315,2019-06-30 09:59:23.120,36.126594,-86.788359,PoweredT7YVE7XBBP2LB,56.0,3,2019-06-30,09:59:23.120000,9
27511316,2019-06-30 09:59:23.120,36.123461,-86.775566,PoweredTC2NFR6OUH7R6,87.0,3,2019-06-30,09:59:23.120000,9
27511317,2019-06-30 09:59:23.120,36.126608,-86.790260,PoweredTOY2TGBXA7QSY,82.0,3,2019-06-30,09:59:23.120000,9


#NOW TO LOOK AT THE TRIPS DF

In [18]:
junetrip.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt
0,2019-06-01 00:05:46.817000,Bird,BRD3572,PoweredSPI1T,4.0,328.084000,2019-06-01 00:00:00,00:02:18.203333,2019-06-01 00:00:00,00:06:16.406666,36.164400,-86.78070,36.163600,-86.780200,"[[36.164679,-86.781089],[36.163693,-86.78011],...",2019-06-02 05:30:19.960000
1,2019-06-01 00:05:46.817000,Bird,BRD3571,Powered2I3MS,5.0,4921.260000,2019-06-01 00:00:00,00:02:44.803333,2019-06-01 00:00:00,00:07:28.286666,36.175300,-86.79430,36.175300,-86.794300,"[[36.175367,-86.794232],[36.175367,-86.794232]...",2019-06-02 05:30:19.927000
2,2019-06-01 00:09:54,Gotcha,GOT1,Powered327,12.0,12.426575,2019-06-01 00:00:00,00:09:56,2019-06-01 00:00:00,00:21:56,36.161501,-86.77601,36.152529,-86.783742,"[[""36.16149"",""-86.77605000000001""]]",2019-06-06 22:23:08.673000
3,2019-06-01 00:10:46.957000,Bird,BRD3610,Powered8U1A6,2.0,0.000000,2019-06-01 00:00:00,00:10:31.163333,2019-06-01 00:00:00,00:12:02.773333,36.164000,-86.78070,36.163100,-86.779700,"[[36.163168,-86.779639]]",2019-06-02 05:30:20.283000
4,2019-06-01 00:10:46.957000,Bird,BRD3612,PoweredSPI1T,5.0,656.168000,2019-06-01 00:00:00,00:07:21.430000,2019-06-01 00:00:00,00:12:30.913333,36.165000,-86.77990,36.165900,-86.777800,"[[36.164951,-86.779836],[36.16494,-86.779456],...",2019-06-02 05:30:20.347000


In [19]:
junetrip.tail()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt
205622,2019-07-01 19:58:07,JUMP,JMP56,Powered2c1eeca5-26f4-5283-bbcd-5f46b19b7681,1440.0,40708.8,2019-06-30 00:00:00,02:16:21,2019-07-01 00:00:00,14:04:18,36.153070,-86.78393,36.160503,-86.77805,"[[""36.153068"",""-86.783928""],[""36.153063"",""-86....",2019-07-02 07:58:39.613000
205623,2019-07-01 19:58:07,JUMP,JMP53,Poweredac7200ea-c5a4-5362-94a8-03e4d2255104,1440.0,36854.4,2019-06-29 00:00:00,21:46:34,2019-07-01 00:00:00,14:04:09,36.162140,-86.77515,36.166332,-86.76144,"[[""36.160706"",""-86.773443""],[""36.160706"",""-86....",2019-07-02 07:58:39.740000
205624,2019-07-01 19:58:07,JUMP,JMP52,Powered3f96a82d-9641-58b6-b6db-b80f746a24cd,1440.0,62356.8,2019-06-29 00:00:00,16:15:42,2019-07-01 00:00:00,14:04:05,36.156837,-86.78104,36.149980,-86.76131,"[[""36.156838"",""-86.781036""],[""36.156845"",""-86....",2019-07-02 07:58:39.827000
205625,2019-07-01 19:58:07,JUMP,JMP54,Poweredc2d0a425-86dd-5664-a036-d770559391bb,1440.0,44880.0,2019-06-29 00:00:00,22:09:51,2019-07-01 00:00:00,14:04:12,36.152622,-86.79712,36.140900,-86.78785,"[[""36.152695"",""-86.79713""],[""36.152695"",""-86.7...",2019-07-02 07:58:39.707000
205626,2019-07-01 19:58:07,JUMP,JMP50,Powered1b56f557-adce-5c68-a98b-ab1ef41c7dcc,1440.0,32472.0,2019-06-29 00:00:00,15:55:36,2019-07-01 00:00:00,14:03:58,36.164547,-86.77692,36.180170,-86.74800,"[[""36.164546"",""-86.776913""],[""36.164546"",""-86....",2019-07-02 07:58:39.893000


In [20]:
junetrip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205627 entries, 0 to 205626
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   pubTimeStamp    205627 non-null  object 
 1   companyName     205627 non-null  object 
 2   tripRecordNum   205627 non-null  object 
 3   sumdID          205627 non-null  object 
 4   tripDuration    205627 non-null  float64
 5   tripDistance    205627 non-null  float64
 6   startDate       205627 non-null  object 
 7   startTime       205627 non-null  object 
 8   endDate         205627 non-null  object 
 9   endTime         205627 non-null  object 
 10  startLatitude   205627 non-null  float64
 11  startLongitude  205627 non-null  float64
 12  endLatitude     205627 non-null  float64
 13  endLongitude    205627 non-null  float64
 14  tripRoute       205627 non-null  object 
 15  create_dt       205627 non-null  object 
dtypes: float64(6), object(10)
memory usage: 25.1+ MB


In [21]:
junetrip.companyName.unique()

array(['Bird', 'Gotcha', 'SPIN', 'Bolt Mobility', 'Lime', 'Lyft', 'JUMP'],
      dtype=object)

In [22]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'SPIN': 4, 'JUMP': 5, 'Bolt Mobility': 6}
junetrip.companyName = junetrip.companyName.replace(company_dict)

In [23]:
junetrip.columns

Index(['pubTimeStamp', 'companyName', 'tripRecordNum', 'sumdID',
       'tripDuration', 'tripDistance', 'startDate', 'startTime', 'endDate',
       'endTime', 'startLatitude', 'startLongitude', 'endLatitude',
       'endLongitude', 'tripRoute', 'create_dt'],
      dtype='object')

In [24]:
junetrip = junetrip.rename(columns={'pubTimeStamp':'pubdatetime', 'companyName':'companyname', 'tripRecordNum':'tripnum', 'sumdID':'sumdid','tripDuration':'tripduration', 'tripDistance':'tripdistance', 'startDate':'startdate', 'startTime':'starttime', 'endDate':'enddate', 'endTime':'endtime', 'startLatitude':'startlat', 'startLongitude':'startlong', 'endLatitude':'endlat', 'endLongitude':'endlong', 'tripRoute':'triproute'})
junetrip.head()

,pubdatetime,companyname,tripnum,sumdid,tripduration,tripdistance,startdate,starttime,enddate,endtime,startlat,startlong,endlat,endlong,triproute,create_dt
0,2019-06-01 00:05:46.817000,0,BRD3572,PoweredSPI1T,4.0,328.084000,2019-06-01 00:00:00,00:02:18.203333,2019-06-01 00:00:00,00:06:16.406666,36.164400,-86.78070,36.163600,-86.780200,"[[36.164679,-86.781089],[36.163693,-86.78011],...",2019-06-02 05:30:19.960000
1,2019-06-01 00:05:46.817000,0,BRD3571,Powered2I3MS,5.0,4921.260000,2019-06-01 00:00:00,00:02:44.803333,2019-06-01 00:00:00,00:07:28.286666,36.175300,-86.79430,36.175300,-86.794300,"[[36.175367,-86.794232],[36.175367,-86.794232]...",2019-06-02 05:30:19.927000
2,2019-06-01 00:09:54,2,GOT1,Powered327,12.0,12.426575,2019-06-01 00:00:00,00:09:56,2019-06-01 00:00:00,00:21:56,36.161501,-86.77601,36.152529,-86.783742,"[[""36.16149"",""-86.77605000000001""]]",2019-06-06 22:23:08.673000
3,2019-06-01 00:10:46.957000,0,BRD3610,Powered8U1A6,2.0,0.000000,2019-06-01 00:00:00,00:10:31.163333,2019-06-01 00:00:00,00:12:02.773333,36.164000,-86.78070,36.163100,-86.779700,"[[36.163168,-86.779639]]",2019-06-02 05:30:20.283000
4,2019-06-01 00:10:46.957000,0,BRD3612,PoweredSPI1T,5.0,656.168000,2019-06-01 00:00:00,00:07:21.430000,2019-06-01 00:00:00,00:12:30.913333,36.165000,-86.77990,36.165900,-86.777800,"[[36.164951,-86.779836],[36.16494,-86.779456],...",2019-06-02 05:30:20.347000


In [25]:
junetrip.pubdatetime = pd.to_datetime(junetrip.pubdatetime)
junetrip['dates'] = pd.to_datetime(junetrip['pubdatetime']).dt.date
junetrip['times'] = pd.to_datetime(junetrip['pubdatetime']).dt.time
junetrip['hours'] = pd.to_datetime(junetrip['pubdatetime']).dt.hour
junetrip.head()

,pubdatetime,companyname,tripnum,sumdid,tripduration,tripdistance,startdate,starttime,enddate,endtime,startlat,startlong,endlat,endlong,triproute,create_dt,dates,times,hours
0,2019-06-01 00:05:46.817,0,BRD3572,PoweredSPI1T,4.0,328.084000,2019-06-01 00:00:00,00:02:18.203333,2019-06-01 00:00:00,00:06:16.406666,36.164400,-86.78070,36.163600,-86.780200,"[[36.164679,-86.781089],[36.163693,-86.78011],...",2019-06-02 05:30:19.960000,2019-06-01,00:05:46.817000,0
1,2019-06-01 00:05:46.817,0,BRD3571,Powered2I3MS,5.0,4921.260000,2019-06-01 00:00:00,00:02:44.803333,2019-06-01 00:00:00,00:07:28.286666,36.175300,-86.79430,36.175300,-86.794300,"[[36.175367,-86.794232],[36.175367,-86.794232]...",2019-06-02 05:30:19.927000,2019-06-01,00:05:46.817000,0
2,2019-06-01 00:09:54.000,2,GOT1,Powered327,12.0,12.426575,2019-06-01 00:00:00,00:09:56,2019-06-01 00:00:00,00:21:56,36.161501,-86.77601,36.152529,-86.783742,"[[""36.16149"",""-86.77605000000001""]]",2019-06-06 22:23:08.673000,2019-06-01,00:09:54,0
3,2019-06-01 00:10:46.957,0,BRD3610,Powered8U1A6,2.0,0.000000,2019-06-01 00:00:00,00:10:31.163333,2019-06-01 00:00:00,00:12:02.773333,36.164000,-86.78070,36.163100,-86.779700,"[[36.163168,-86.779639]]",2019-06-02 05:30:20.283000,2019-06-01,00:10:46.957000,0
4,2019-06-01 00:10:46.957,0,BRD3612,PoweredSPI1T,5.0,656.168000,2019-06-01 00:00:00,00:07:21.430000,2019-06-01 00:00:00,00:12:30.913333,36.165000,-86.77990,36.165900,-86.777800,"[[36.164951,-86.779836],[36.16494,-86.779456],...",2019-06-02 05:30:20.347000,2019-06-01,00:10:46.957000,0


REPEATING WORK FROM MAY INTO JUNE NOTEBOOK TO GATHER ABANDONED SCOOTER DATA

In [26]:
junesorted = june.sort_values(['sumdid', 'pubdatetime'])

In [27]:
junesorted['latmovement'] = (junesorted.latitude.shift() - junesorted.latitude)

In [28]:
june_stationaryscooter= junesorted[junesorted['latmovement']==0.0]

In [29]:
june_stationaryscooter['lonmovement'] = (june_stationaryscooter.longitude.shift() - june_stationaryscooter.longitude)

<ipython-input-29-8851fe749052>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  june_stationaryscooter['lonmovement'] = (june_stationaryscooter.longitude.shift() - june_stationaryscooter.longitude)


In [30]:
june_stationaryscooter = june_stationaryscooter[june_stationaryscooter['lonmovement']==0.0]

In [31]:
june_stationaryscooter.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,dates,times,hours,latmovement,lonmovement
255360,2019-06-01 06:42:14,36.152751,-86.769299,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,56.64,6,2019-06-01,06:42:14,6,0.0,0.0
268534,2019-06-01 07:07:15,36.152744,-86.769321,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,56.64,6,2019-06-01,07:07:15,7,0.0,0.0
553942,2019-06-01 14:52:32,36.152765,-86.769309,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,54.65,6,2019-06-01,14:52:32,14,0.0,0.0
558052,2019-06-01 14:57:33,36.152765,-86.769309,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,54.65,6,2019-06-01,14:57:33,14,0.0,0.0
966392,2019-06-01 23:47:53,36.152744,-86.769319,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,53.51,6,2019-06-01,23:47:53,23,0.0,0.0


In [32]:
june_stationaryscooter['timeshift']= (june_stationaryscooter.pubdatetime - june_stationaryscooter.pubdatetime.shift())

In [33]:
june_stationaryscooter['timeshift'] = june_stationaryscooter['timeshift']/np.timedelta64(1, 's')

In [34]:
def duration(row): 
    if 240 <= row ['timeshift'] <= 360:
        return 5
    else:
        return 0

june_stationaryscooter['duration'] = june_stationaryscooter.apply(duration, axis = 1)

In [35]:
june_stationaryscooter.head(10)

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,dates,times,hours,latmovement,lonmovement,timeshift,duration
255360,2019-06-01 06:42:14,36.152751,-86.769299,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,56.64,6,2019-06-01,06:42:14,6,0.0,0.0,NaN,0
268534,2019-06-01 07:07:15,36.152744,-86.769321,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,56.64,6,2019-06-01,07:07:15,7,0.0,0.0,1501.0,0
553942,2019-06-01 14:52:32,36.152765,-86.769309,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,54.65,6,2019-06-01,14:52:32,14,0.0,0.0,27917.0,0
558052,2019-06-01 14:57:33,36.152765,-86.769309,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,54.65,6,2019-06-01,14:57:33,14,0.0,0.0,301.0,5
966392,2019-06-01 23:47:53,36.152744,-86.769319,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,53.51,6,2019-06-01,23:47:53,23,0.0,0.0,31820.0,0
1260373,2019-06-02 07:58:12,36.152744,-86.769320,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,51.81,6,2019-06-02,07:58:12,7,0.0,0.0,29419.0,0
1262822,2019-06-02 08:03:12,36.152744,-86.769320,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,51.81,6,2019-06-02,08:03:12,8,0.0,0.0,300.0,5
1381090,2019-06-02 11:58:20,36.152769,-86.769299,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,50.68,6,2019-06-02,11:58:20,11,0.0,0.0,14108.0,0
1513178,2019-06-02 15:13:29,36.152765,-86.769290,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,49.54,6,2019-06-02,15:13:29,15,0.0,0.0,11709.0,0
1516722,2019-06-02 15:18:29,36.152765,-86.769290,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,49.54,6,2019-06-02,15:18:29,15,0.0,0.0,300.0,5


In [36]:
june_stationaryscooter.to_pickle("../data/june_stationaryscooter.pkl")